# Creating Synthetic Data

If you want to adapt a LLm as per your own domain,use case, custom data:
* then you need to fine tune it

fine-tuning a model requires:
* a quality dataset

you want to train a model on your company's data:
* company data just cannot be fed into a model
* the data could be in any format, it could be PDF, it cluld be text, it could be in CSV format, or list goes on and on ...

if your model has been trained garbage:
* garbage in garbage out
* make sure that you spend as much time on your data as possible.

implementing these llms on your own specific domain:
* is the acquiring of good data set
* creating a dataset by hand tales time
* there are lot of steps involved in that  

## distilabel
* allows you create a synthetic dataset from scratch
* you don't require any GPU, any API cost
* iti is primarily a framework for synthetic data generation
* if you require high quality outputs and dataset with full ownership privately
* it is very good.
* you are working on a predictive model that computes semantic similarity
* or the next generative model that is going to beat llm benchmarks
* this distilabel framework ensures that the hard data work pays off

## Installing
* go to huggingface website, log in, access token , get a token
* in huggingface, create a new dataset,

#### For now
* give a very simple data
* it is a array wheere there are two items

In [ ]:
DATA = [
    "These wireless earbuds have amazing sound quality and a long-lasting battery, making them a great purchase."
    "The fitness tracker is incredibly accurate and has motivated me to stay active every day."
    "This smart home speaker has fantastic voice recognition and sound quality, making it a stylish and functional addition to my home."
]

install the distilabel

In [ ]:
! pip install --upgrade -qqq git+https://github.com/argilla-io/distilabel.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from distilabel.llms.huggingface import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps.generators.data import LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration
from distilabel.steps.globals.huggingface import PushToHub
from google.colab import userdata

In [ ]:
REPO_ID = "eminesener/synthetic_data_exp"

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get("rag")

In [ ]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TEMPERATURE = 0.7
MAX_TOKENS = 512

### Temperature
* which will show the creativity of the model
* We have kept 0.7 bit higher
* because we are creating a dataset
* we  want model ta be creative
* you can increase it if you want   

In [ ]:
with Pipeline(
    name= "synthetic_data_exp_pipeline",
    description = "A simple DPO dataste generation pipeline",
) as pipeline:
  llm = InferenceEndpointsLLM(
      base_url = f"https://api-inference.huggingface.co/models{MODEL_ID}",
      api_key = HF_TOKEN
  )

  load_data = LoadDataFromDicts(
      name="load_data",
      data=[
          {
              "instruction":sample,
          }for sample in DATA

      ],
      batch_size = 1,
  )
  generate_with_mistral = TextGeneration(
      name="generate_with_mistral", llm=llm
  )

  load_data.connect(generate_with_mistral)

In [ ]:
distiset = pipeline.run(
    parameters={
        "generate_with_mistral":{
            "llm": {
                "generation_kwargs":{
                    "temperature":TEMPERATURE,
                    "max_new_tokens":MAX_TOKENS,
                    "num_generations":2
                }
            }
        }
    },
)

[08/04/24 09:56:15] INFO     ['distilabel.pipeline'] 💾 Loading `_BatchManager` from cache:             ]8;id=911153;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=297049;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#599\599]8;;\
                             '/root/.cache/distilabel/pipelines/synthetic_data_exp_pipeline/dbbd8bd33ab            
                             ca5e80435aca249e2ec3931bb8605/batch_manager.json'                                     

                    INFO     ['distilabel.pipeline'] 💾 Loaded batch manager from cache doesn't contain ]8;id=604115;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=37852;file:///usr/local/lib/python3.10/dist-packages/distilabel/pipeline/base.py#352\352]8;;\
                             any remaining data. Returning `Distiset` from cache data...                           

In [ ]:
distiset

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['instruction', 'generation', 'model_name'],
            num_rows: 1
        })
    })
})

In [ ]:
import os
from google.colab import auth
auth.authenticate_user()

# API anahtarınızı bir çevresel değişken olarak ayarlama
os.environ['rag'] = ''


In [ ]:
import os
from huggingface_hub import login

# API anahtarını kullanarak Hugging Face'e giriş yapma
login(token=os.environ['rag'])

# `distiset`'i kullanarak yükleme işlemini yapabilirsiniz
distiset.push_to_hub(REPO_ID)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
distiset

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['instruction', 'generation', 'model_name'],
            num_rows: 1
        })
    })
})